In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import smtplib
import os
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import random
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import time
import math
import difflib
import requests
import os
import cv2  # Import OpenCV
import numpy as np
from PIL import Image
# Specify the path to the geckodriver executable
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from PIL import Image
import numpy as np
import openpyxl
from openpyxl.drawing.image import Image
from openpyxl.styles import Font, PatternFill


amazon_titles= []
amazon_links = []
webuy_titles = []
webuy_links = []
amazon_images = []
webuy_images = []
calculated_prices = []
product_codes = []
buy_prices = []

with open("WorkingProxies.txt", "r", encoding="utf-8") as prxsfile:
    proxy_list = prxsfile.read().split("\n")
def get_random_proxy():
    """Select a random proxy from the list."""
    return random.choice(proxy_list)

def create_driver_with_proxy(proxy):
    """Create a new Chrome WebDriver instance with a proxy."""
    # Split the proxy IP and port
    proxy_ip, proxy_port = proxy.split(':')
    
    # Create Chrome options
    chrome_options = Options()
    
    # Configure proxy settings for both HTTP and HTTPS
    proxy_settings = f"{proxy_ip}:{proxy_port}"
    chrome_options.add_argument(f'--proxy-server={proxy_settings}')
    
    # Specify the path to the Chrome executable if needed (optional)
    # chrome_options.binary_location = "C:\\Path\\To\\Chrome.exe"  # Only needed if Chrome is not in PATH
    
    # Initialize the WebDriver with the options
    amazon_driver = webdriver.Chrome(options=chrome_options)
    return amazon_driver

def create_driver_without_proxy():
    """Create a new Chrome WebDriver instance without a proxy."""

    
    # Initialize the WebDriver with the options
    driver = webdriver.Chrome()
    return driver
def wait_for_element_to_be_clickable(driver, xpath, timeout=10):
    return WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((By.XPATH, xpath))
    )

def check_blacklisted_sellers(sellers_list, sellers_black_list):
    # Convert the blacklist to lowercase and strip whitespace
    stripped_blacklist = [blacklisted_seller.strip().lower() for blacklisted_seller in sellers_black_list]

    # Find the blacklisted sellers in the sellers list, ignoring case and whitespace
    blacklisted_sellers = [
        seller for seller in sellers_list
        if seller.strip().lower() in stripped_blacklist
    ]

    if blacklisted_sellers:
        print("Not a valid product")
        return True
    else:
        print("No sellers in the list are blacklisted.")
        return False


def check_whitelisted_sellers(sellers_list, seller_white_list):
    # Convert the whitelist to lowercase and strip whitespace
    stripped_whitelist = [whitelist_seller.strip().lower() for whitelist_seller in seller_white_list]

    # Loop through the sellers list to find the first whitelisted seller
    for index, seller in enumerate(sellers_list):
        if seller.strip().lower() in stripped_whitelist:
            print("First whitelisted seller found at index:", index)
            return index  # Return the index of the first whitelisted seller

    print("No sellers in the list are whitelisted.")
    return -1  # Return -1 if no whitelisted seller is found

def calculate_percentages(title_data, image_data):
    results = []
    
    # Ensure both lists have the same length
    if len(title_data) != len(image_data):
        raise ValueError("The length of title_data and image_data must be the same.")
    
    for (title, title_percentage), (image, image_percentage) in zip(title_data, image_data):
        new_percentage = (0.8 * image_percentage) + (0.2 * title_percentage)
        results.append(new_percentage)
    
    return results

def calculate_similarity(reference_string, string_list):
    similarity_scores = []
    
    for string in string_list:
        # Calculate similarity ratio
        similarity_ratio = difflib.SequenceMatcher(None, reference_string, string).ratio()
        # Convert to percentage
        similarity_percentage = similarity_ratio * 100
        similarity_scores.append((string, similarity_percentage))
    
    return similarity_scores

def download_reference_image(url, download_dir='images\\reference'):
    """Download the reference image to a specified directory."""
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses
        image_name = os.path.join(download_dir, url.split("/")[-1])
        with open(image_name, 'wb') as file:
            file.write(response.content)
        return image_name
    except Exception as e:
        print(f"Error downloading reference image {url}: {e}")
        return None




def clear_temp_images(download_dir='images\\temporary_images'):
    """Clear all images in the temporary images directory."""
    if os.path.exists(download_dir):
        for filename in os.listdir(download_dir):
            file_path = os.path.join(download_dir, filename)
            try:
                if os.path.isfile(file_path):
                    
                    os.remove(file_path)  # Remove the file
                    print(f"Deleted: {file_path}")
            except Exception as e:
                print(f"Error deleting {file_path}: {e}")
    else:
        print(f"The directory {download_dir} does not exist.")

def download_images(image_urls, download_dir='images\\temporary_images'):
    """Download images from the given URLs."""
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)

    downloaded_images = []
    for url in image_urls:
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an error for bad responses
            image_name = os.path.join(download_dir, url.split("/")[-1])
            with open(image_name, 'wb') as file:
                file.write(response.content)
            downloaded_images.append(image_name)
        except Exception as e:
            print(f"Error downloading {url}: {e}")

    return downloaded_images

def check_black_list_words(word_list, target_string):
    # Convert the target string to lowercase
    target_string = target_string.lower().strip()

    # Loop through each word in the list and check in lowercase
    for word in word_list:
        if word.lower().strip() in target_string:
            return True
    return False

def check_prices_more_than_webuyPrice(amazon_prices, webuyprice):
    # Initialize a counter for numbers less than the threshold
    count = 0

    # Loop through each number in the list
    for number in amazon_prices:
        if number < webuyprice:
            count += 1
        # If the count reaches 3, return True immediately
        if count >= 3:
            return True

    # If the loop finishes and less than 3 numbers are found, return False
    return False


sellers_black_list = [" Live Life Retail"]
seller_white_list = [" Selling-Prime-UK"]
words_black_list =  ["ntsc", "renewed", "reproduction"]





    

In [2]:

# Rotate proxies
high_price = 30
low_price = 25
proxy = get_random_proxy()  # Get a random proxy
#driver = create_driver_with_proxy(proxy)  # Create WebDriver with that proxy
amazon_driver = create_driver_without_proxy()
url = f"https://www.amazon.co.uk/"
try:
    amazon_driver.get(url)
except Exception:
    pass
amazon_cookies_PATH = '/html/body/div[2]/span/form/div[2]/span[1]/span/input'
try:
    # Wait until the element is clickable
    WebDriverWait(amazon_driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, amazon_cookies_PATH))
    )
    amazon_driver.find_element(By.XPATH, amazon_cookies_PATH).click()
except Exception as e:
    print(f"An error occurred: {e}")
    time.sleep(0.5)

# Click the "Deliver to" button
deliver_to_button_PATH = '//*[@id="glow-ingress-line2"]'
wait_for_element_to_be_clickable(amazon_driver, deliver_to_button_PATH).click()

# Click the Post Code input field
PostCode_input_PATH = '//*[@id="GLUXZipUpdateInput"]'
wait_for_element_to_be_clickable(amazon_driver, PostCode_input_PATH).click()

# Enter the postcode
amazon_driver.find_element(By.XPATH, PostCode_input_PATH).send_keys("SW1A2aa")

# Click the "Apply" button
Apply_button_PATH = '//*[@id="GLUXZipUpdate"]/span/input'
wait_for_element_to_be_clickable(amazon_driver, Apply_button_PATH).click()

# Click the "Continue" button
continue_button_PATH = '/html/body/div[5]/div/div/div[2]/span/span/input'
wait_for_element_to_be_clickable(amazon_driver, continue_button_PATH).click()

# Click to accept cookies
accept_cookies_PATH = '/html/body/div[1]/span/form/div[2]/span[1]/span/input'
wait_for_element_to_be_clickable(amazon_driver, accept_cookies_PATH).click()


An error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7376BB095+29557]
	(No symbol) [0x00007FF73762FA50]
	(No symbol) [0x00007FF7374EB56A]
	(No symbol) [0x00007FF73753F695]
	(No symbol) [0x00007FF73753F8EC]
	(No symbol) [0x00007FF73758B777]
	(No symbol) [0x00007FF7375671CF]
	(No symbol) [0x00007FF73758851C]
	(No symbol) [0x00007FF737566F33]
	(No symbol) [0x00007FF73753116F]
	(No symbol) [0x00007FF7375322D1]
	GetHandleVerifier [0x00007FF7379EC96D+3378253]
	GetHandleVerifier [0x00007FF737A38497+3688311]
	GetHandleVerifier [0x00007FF737A2D1CB+3642539]
	GetHandleVerifier [0x00007FF73777A6B6+813462]
	(No symbol) [0x00007FF73763AB5F]
	(No symbol) [0x00007FF737636B74]
	(No symbol) [0x00007FF737636D10]
	(No symbol) [0x00007FF737625C1F]
	BaseThreadInitThunk [0x00007FFFEB127374+20]
	RtlUserThreadStart [0x00007FFFECE5CC91+33]



TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7376BB095+29557]
	(No symbol) [0x00007FF73762FA50]
	(No symbol) [0x00007FF7374EB56A]
	(No symbol) [0x00007FF73753F695]
	(No symbol) [0x00007FF73753F8EC]
	(No symbol) [0x00007FF73758B777]
	(No symbol) [0x00007FF7375671CF]
	(No symbol) [0x00007FF73758851C]
	(No symbol) [0x00007FF737566F33]
	(No symbol) [0x00007FF73753116F]
	(No symbol) [0x00007FF7375322D1]
	GetHandleVerifier [0x00007FF7379EC96D+3378253]
	GetHandleVerifier [0x00007FF737A38497+3688311]
	GetHandleVerifier [0x00007FF737A2D1CB+3642539]
	GetHandleVerifier [0x00007FF73777A6B6+813462]
	(No symbol) [0x00007FF73763AB5F]
	(No symbol) [0x00007FF737636B74]
	(No symbol) [0x00007FF737636D10]
	(No symbol) [0x00007FF737625C1F]
	BaseThreadInitThunk [0x00007FFFEB127374+20]
	RtlUserThreadStart [0x00007FFFECE5CC91+33]


In [59]:
amazon_titles= []
amazon_links = []
webuy_titles = []
webuy_links = []
amazon_images = []
webuy_images = []
calculated_prices = []
product_codes = []
buy_prices = []

In [49]:
amazon_first_links3 =[]
low_price = 25
high_price = 30
page = 1

while high_price < 255:
    url = f"https://www.amazon.co.uk/s?k=blu+ray&i=merchant-items&me=A1UFLUL80142N4&s=price-desc-rank&page={page}&qid=1728834750&rnid=389127011&ref=sr_nr_p_36_0_0&low-price={low_price}&high-price={high_price}"
    amazon_driver.get(url)

    # Initialize previous range variables
    previous_low_price = low_price
    previous_high_price = high_price

    while True:
        try:
            # Get the number of products
            Num_of_products = int(amazon_driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/span/div/h1/div/div[1]").text.split("results")[0].strip().split(" ")[-1].replace(",", ''))
        except Exception:
            Num_of_products = 0

        # Check if the number of products is greater than 320
        if Num_of_products > 320:
            try:
                time.sleep(3)

                # If low_price and high_price are the same, increment high_price slightly
                if round(low_price, 2) == round(high_price, 2):
                    high_price += 0.01
                    break

                # Calculate the new high price
                new_high_price = (low_price + high_price) / 2

                # Ensure new_high_price is still greater than low_price
                if new_high_price <= low_price:
                    new_high_price = low_price + 0.01  # Adjust if needed

                # Build the updated URL with the new high_price
                url = f"https://www.amazon.co.uk/s?k=blu+ray&i=merchant-items&me=A1UFLUL80142N4&s=price-desc-rank&page={page}&qid=1728834750&rnid=389127011&ref=sr_nr_p_36_0_0&low-price={low_price}&high-price={high_price}"

                # Navigate to the updated URL
                amazon_driver.get(url)

                # Re-fetch the number of products
                Num_of_products = int(amazon_driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/span/div/h1/div/div[1]").text.split("results")[0].strip().split(" ")[-1].replace(",", ''))

                # If no products found, revert to the previous range
                if Num_of_products == 0:
                    low_price = previous_low_price
                    high_price = previous_high_price
                    print("No products found, reverting to previous range.")
                    break  # Exit the inner loop to retry with the previous range
                else:
                    # Update previous range for the next iteration
                    previous_low_price = low_price
                    previous_high_price = high_price
                    high_price = new_high_price  # Update to the new high price

            except Exception as e:
                 # Update previous range for the next iteration
                previous_low_price = low_price
                previous_high_price = high_price
                high_price = new_high_price  # Update to the new high price
                print(f"An error occurred: {e}")
                break  # Exit the inner loop on error
        else:
            break  # If the number of products is 320 or less, exit the loop

    print(f"Final high price: {high_price}, Final low price: {low_price}, Number of products: {Num_of_products}")

    # Pagination handling
    pages = 20 if Num_of_products / 16 >= 20 else math.ceil(Num_of_products / 16)
    print(f"Number of pages: {pages}")

    for i in range(1, pages + 1):
        if i != 1:
            url = f"https://www.amazon.co.uk/s?k=blu+ray&i=merchant-items&me=A1UFLUL80142N4&s=price-desc-rank&page={i}&qid=1728834750&rnid=389127011&ref=sr_nr_p_36_0_0&low-price={low_price}&high-price={high_price}"
            amazon_driver.get(url)

        # Scroll down to load more products
        last_height = amazon_driver.execute_script("return document.body.scrollHeight")
        tries = 20  # Number of tries before stopping

        for j in range(tries):
            try:
                time.sleep(1)  # Wait for page to load
                amazon_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # Scroll to the bottom
                new_height = amazon_driver.execute_script("return document.body.scrollHeight")

                if new_height == last_height:  # Check if we've reached the end
                    break  # Exit the scrolling loop if no more content is loading
                
                last_height = new_height  # Update the last height
            except Exception as error:
                print(error)

        # Parse the page content
        html_source = amazon_driver.page_source
        soup = BeautifulSoup(html_source, "html.parser")
        products_link_tags = soup.find_all(class_="a-link-normal s-no-outline")
        amazon_products_links = ["https://www.amazon.co.uk/" + products_link['href'] for products_link in products_link_tags]
        
        amazon_first_links3.extend(amazon_products_links)  # Append new links to the list
        print(f"Page {i} processed. Number of product links found: {len(amazon_products_links)}")
    
    # Prepare for the next price range
    temp_price = low_price
    low_price = high_price
    high_price += 5  # Increment the high price for the next iteration
    

# Print or process the gathered product links
print(f"Total product links gathered: {len(amazon_first_links3)}")

Final high price: 25.625, Final low price: 25, Number of products: 71
Number of pages: 5
Page 1 processed. Number of product links found: 16
Page 2 processed. Number of product links found: 16
Page 3 processed. Number of product links found: 1
Page 4 processed. Number of product links found: 0
Page 5 processed. Number of product links found: 0
Final high price: 26.25, Final low price: 25.625, Number of products: 320
Number of pages: 20
Page 1 processed. Number of product links found: 16
Page 2 processed. Number of product links found: 16
Page 3 processed. Number of product links found: 6
Page 4 processed. Number of product links found: 0
Page 5 processed. Number of product links found: 0
Page 6 processed. Number of product links found: 0
Page 7 processed. Number of product links found: 0
Page 8 processed. Number of product links found: 0
Page 9 processed. Number of product links found: 0
Page 10 processed. Number of product links found: 0
Page 11 processed. Number of product links foun

In [4]:
with open("all_output_links.txt", "r") as file:
    amazon_first_links = file.readlines()

In [5]:
amazon_first_links[0]


'https://www.amazon.co.uk//Shogun-Assassin-Blu-ray-Tomisaburo-Wakayama/dp/B004G6TD7U/ref=sr_1_1?dib=eyJ2IjoiMSJ9.cqrAcr2-NpXDchP6XOKUNxlYoA1YUTz5Hd-tkr4T1j-_1iEbqXwc0f98NkdhkAxL4yZonhPNRai9jsGYdeVCVJZv9SW6ZzosgMu1nOVN5wA0txc9AK5izVAo-1sfjxnI0vUUOnmEF1qcHSPNEgNly-hKbBOaAO_P64c4FS5ibbMIVA3eTIe2VYHspWuCZQFgIxpbT3F997cb8y4e2951VCLfx4yB_vSVUQ2_39Vhq00.asZxE3LVJw7uygAbmEzIz3RD2XR4v4EK-wzcC0OyQk4&dib_tag=se&keywords=blu+ray&m=A1UFLUL80142N4&nsdOptOutParam=true&qid=1728835033&refinements=p_36%3A2500-2625&rnid=389127011&s=merchant-items&sr=1-1\n'

In [43]:
webuy_driver = create_driver_without_proxy()

In [6]:

amazon_titles= []
amazon_links = []
webuy_titles = []
webuy_links = []
amazon_images = []
webuy_images = []
calculated_prices = []
product_codes = []
buy_prices = []

In [10]:

def calculate_ssim(img1, img2):
    """Calculate SSIM manually."""
    # Constants to avoid division by zero
    C1 = 6.5025
    C2 = 58.5225

    # Calculate means
    mu1 = np.mean(img1)
    mu2 = np.mean(img2)

    # Calculate variances
    sigma1_sq = np.var(img1)
    sigma2_sq = np.var(img2)
    sigma12 = np.cov(img1.flatten(), img2.flatten())[0][1]

    # SSIM formula
    ssim = ((2 * mu1 * mu2 + C1) * (2 * sigma12 + C2)) / \
            ((mu1 ** 2 + mu2 ** 2 + C1) * (sigma1_sq + sigma2_sq + C2))

    return ssim

def compare_images(image1_path, image2_path):
    """Compare two images and return similarity percentage using OpenCV."""
    # Read the images
    img1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
    img2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale

    # Check if either image failed to load
    if img1 is None or img2 is None:
        print(f"Error: One of the images could not be loaded: {image1_path}, {image2_path}")
        return 0  # Return 0 if an image could not be loaded

    # Resize images to the same size for comparison
    img1 = cv2.resize(img1, (100, 100))  # Adjust size as necessary
    img2 = cv2.resize(img2, (100, 100))  # Adjust size as necessary

    # Calculate SSIM
    ssim_score = calculate_ssim(img1, img2)

    # Convert to percentage
    similarity_percentage = ssim_score * 100
    return similarity_percentage

def download_webuy_image(url, download_dir='images\\webuy_images'):
    """Download the reference image to a specified directory."""
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses
        image_name = os.path.join(download_dir, url.split("/")[-1])
        with open(image_name, 'wb') as file:
            file.write(response.content)
        return image_name
    except Exception as e:
        print(f"Error downloading reference image {url}: {e}")
        return None


def download_webuy_image(url, download_dir='images\\webuy_images'):
    """Download the reference image to a specified directory."""
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses
        image_name = os.path.join(download_dir, url.split("/")[-1])
        with open(image_name, 'wb') as file:
            file.write(response.content)
        return image_name
    except Exception as e:
        print(f"Error downloading reference image {url}: {e}")
        return None



for k in range(8772, 8850):
    link = amazon_first_links[k]
    valid_product = True
    while valid_product == True:
        try:
            amazon_driver.get(link)

            product_code = link.split("/dp/")[1].split("/")[0] 





            other_sellers_button1 = '/html/body/div[2]/div/div[5]/div[4]/div[1]/div[5]/div[2]/div/div[2]/a/span'
            other_sellers_button2 = 'buybox-see-all-buying-choices'
            other_sellers_button3 = "/html/body/div[3]/div/div[8]/div[4]/div[1]/div[5]/div[2]/div/div[2]/a/span/span[1]"
            other_sellers_button4 = "a-icon a-icon-arrow a-icon-small daodi-arrow-icon"
            section_opened = False
            other_sellers_button_PATH = 'a-price'

            outofstock = False
            for i in range(0,3):
                try: 
                    html_source = amazon_driver.page_source
                    soup = BeautifulSoup(html_source, "html.parser")
                    check_stock = soup.find(id="outOfStock")
                    if check_stock:
                        outofstock = True
                        break
                    else:
                        time.sleep(1)
                except Exception as e:
                    print(e)
            if outofstock:
                print("out")
                valid_product = False
                continue

            element_found = False
            try:
                # Wait until the element is clickable
                WebDriverWait(amazon_driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, other_sellers_button1))
                )
                amazon_driver.find_element(By.XPATH, other_sellers_button1).click()
                print("elemet found")
                element_found = True
                time.sleep(1)
                for i in range(0,3):
                        try: 
                            html_source = amazon_driver.page_source
                            soup = BeautifulSoup(html_source, "html.parser")
                            slider = soup.find(id="all-offers-display-scroller")
                            if slider:
                                section_opened = True
                                break
                            else:
                                time.sleep(1)
                        except Exception as e:
                            pass
            except Exception as e:
                element_found = False
                print(f"Element not found")
                time.sleep(0.5)

            if not element_found:
                other_sellers_button_js_path = 'document.querySelector("#dynamic-aod-ingress-box > div > div.a-section.a-spacing-none.daodi-content > a > span > span:nth-child(1)")'

                try:
                    # Use JavaScript to find and click the element
                    amazon_driver.execute_script(f'{other_sellers_button_js_path}.click();')
                    print("Element clicked via JS path")
                    element_found = True
                    time.sleep(1)
                    
                    for i in range(3):
                        try:
                            html_source = amazon_driver.page_source
                            soup = BeautifulSoup(html_source, "html.parser")
                            slider = soup.find(id="all-offers-display-scroller")
                            if slider:
                                section_opened = True
                                break
                            else:
                                time.sleep(1)
                        except Exception as e:
                            pass
                except Exception as e:
                    element_found = False
                    print(f"Element not found or not clickable via JS path: {e}")
                    time.sleep(0.5)


            if not element_found:
                try:
                    # Wait until the element is clickable
                    WebDriverWait(amazon_driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, other_sellers_button3))
                    )
                    amazon_driver.find_element(By.XPATH, other_sellers_button3).click()
                    print("elemet found")
                    element_found = True
                    time.sleep(1)
                    for i in range(0,3):
                            try: 
                                html_source = amazon_driver.page_source
                                soup = BeautifulSoup(html_source, "html.parser")
                                slider = soup.find(id="all-offers-display-scroller")
                                if slider:
                                    section_opened = True
                                    break
                                else:
                                    time.sleep(1)
                            except Exception as e:
                                pass
                except Exception as e:
                    element_found = False
                    print(f"Element not found")
                    time.sleep(0.5)
            if not section_opened:
                try:
                        
                    # Wait until the element is clickable
                    WebDriverWait(amazon_driver, 10).until(
                        EC.element_to_be_clickable((By.ID, other_sellers_button2))
                    )
                    amazon_driver.find_element(By.ID, other_sellers_button2).click()
                    print("elemet found")
                    
                    section_opened = True
                    time.sleep(1)
                except Exception:
                    section_opened = False
                    time.sleep(1)
                    print("Element not found")



            time.sleep(1)
            if section_opened:
                for i in range(0,3):
                        try: 
                            html_source = amazon_driver.page_source
                            soup = BeautifulSoup(html_source, "html.parser")
                            slider = soup.find(id="all-offers-display-scroller")
                            if slider:
                                section_opened = True
                                break
                            else:
                                time.sleep(1)
                        except Exception as e:
                            pass
            else:

                html_source = amazon_driver.page_source
                soup = BeautifulSoup(html_source, "html.parser")
                amazon_sellertag = soup.find(id="sellerProfileTriggerId")
                amazon_pricetag = soup.find(class_="a-size-base a-color-price offer-price a-text-normal")
                if amazon_sellertag and amazon_pricetag:
                    amazon_price = float(amazon_pricetag.text.strip().replace("£", ""))
                    for seller in seller_white_list:

                        if amazon_sellertag.text.strip() not in seller_white_list:
                            valid_product = False
                            break
                    if not valid_product:
                        print("invalid product1")
                        continue
                else:
                    print("invalid product2")
                    valid_product = False
                    continue


            if section_opened:
                for i in range(0,5):
                    try: 
                        html_source = amazon_driver.page_source
                        soup = BeautifulSoup(html_source, "html.parser")
                        slider = soup.find(id="all-offers-display-scroller")
                        if slider:
                            amazon_sellerstags = slider.find_all(class_="a-size-small a-link-normal")
                            amazon_pricestags = slider.find_all(class_="a-section a-spacing-none aok-align-center aok-relative")
                            break
                        else:
                            time.sleep(1)
                    except Exception as e:
                        print(e)



                amazon_sellers = []
                for seller in amazon_sellerstags:
                    try:
                        # Check if seller is valid and has the required aria-label attribute
                        if seller["aria-label"] == "Opens a new page":
                            amazon_sellers.append(seller.text.strip())  # Add to the list if conditions are met
                    except Exception as e:
                        # Handle any exceptions (e.g., element doesn't support get_attribute)
                        print(e)  # You can log the exception if needed or just pass






                # Example usage
                Wseller_index = check_whitelisted_sellers(amazon_sellers, seller_white_list)
                if Wseller_index == -1:
                    print("invalid product3")
                    valid_product = False
                    continue
                if check_blacklisted_sellers(amazon_sellers, sellers_black_list):
                    print("invalid product4")
                    valid_product = False
                    continue
                sellers20pounds_counter = 0
                sellers10pounds_counter = 0

                amazon_prices = []
                for p in amazon_pricestags:
                    priceSTR = p.find(class_='aok-offscreen').text.strip().split(" ")[0].replace("£", "")
                    riceNMR = float(priceSTR.replace("£", "").replace(",", ""))
                    amazon_prices.append(riceNMR)
                    if riceNMR <= 20.00:
                        sellers20pounds_counter += 1
                    if riceNMR <= 10.00:
                        sellers10pounds_counter += 1
                if sellers20pounds_counter > 2 or sellers10pounds_counter >= 2:
                    print("invalid product5")
                    valid_product = False
                    continue
                    
                amazon_price = amazon_prices [Wseller_index]
            


            percentage = 0.01
            # Calculate 1% of the price
            reduction = amazon_price * percentage

            # Calculate the new price
            calculated_price = amazon_price - reduction

            
            print("Price after reducing 1% from", amazon_price, "is", calculated_price)




            # # Removing duplicates while maintaining order using a set
            # unique_sellers = []
            # seen_sellers = set()
            # for seller in sellers:
            #     if seller not in seen_sellers:
            #         unique_sellers.append(seller)
            #         seen_sellers.add(seller)

            # unique_prices = []
            # seen_prices = set()
            # for price in prices:
            #     if price not in seen_prices:
            #         unique_prices.append(price)
            #         seen_prices.add(price)

            # print(unique_sellers)  # This will maintain the original order of sellers
            # print(unique_prices)   # This will maintain the original order of prices

            html_source = amazon_driver.page_source
            soup = BeautifulSoup(html_source, "html.parser")
            product_title = soup.find(id="productTitle").text.strip()
            if check_black_list_words(words_black_list, product_title):
                print("invalid product6")
                valid_product = False
                continue
            
            product_image_tag = soup.find(id='imgTagWrapperId')
            product_image = product_image_tag.find('img')['src']
            

            webuy_url = f"https://uk.webuy.com/search?stext={product_title.replace(' ', '+').replace('&', 'and')}"
            amazon_driver.get(webuy_url)
            time.sleep(5)
            html_source = amazon_driver.page_source
            webuy_soup = BeautifulSoup(html_source, "html.parser")
            for _ in range(0, 5):
                html_source = amazon_driver.page_source
                webuy_soup = BeautifulSoup(html_source, "html.parser")


                cookies = webuy_soup.find(id="onetrust-accept-btn-handler")
                cookies_path = '/html/body/div[3]/div[2]/div/div/div[2]/div/div/button'
                if cookies:
                    print("cookies found")
                    wait_for_element_to_be_clickable(amazon_driver, cookies_path).click()

                    break
                else:
                    time.sleep(1)
            for _ in range(0, 5):


                html_source = amazon_driver.page_source
                webuy_soup = BeautifulSoup(html_source, "html.parser")
                results = webuy_soup.find(class_='search-result-grid mb-s md-mb-0')
                if results:
                    break
                else:
                    time.sleep(1)
            try:
                html_source = amazon_driver.page_source
                webuy_soup = BeautifulSoup(html_source, "html.parser")
                results = webuy_soup.find(class_='search-result-grid mb-s md-mb-0')
            except Exception:
                print("invalid product7")
                valid_product = False
                continue

            webuy_products = results.find_all(class_='search-product-card')

            webuy_product_titles = []
            webuy_product_links = []
            webuy_product_images = []
            for product in webuy_products:
                webuy_product_titles.append(product.find(class_='card-title').text)
                webuy_product_links.append('https://uk.webuy.com' + product.find('a')['href'])
                webuy_product_images.append(product.find('a').find("img")['src'])






            title_similarity_results = calculate_similarity(product_title, webuy_product_titles)

            # Print results
            for string, score in title_similarity_results:
                print(f"Similarity with '{string}': {score:.2f}%")






            print("Libraries are installed correctly!")







            # Download images
            downloaded_images = download_images(webuy_product_images)

            # Download the reference image
            reference_image_path = download_reference_image(product_image)
            
            # Compare each downloaded image with the reference image if it was successfully downloaded
            if reference_image_path:
                image_similarity_results = []
                for img_path in downloaded_images:
                    similarity = compare_images(reference_image_path, img_path)
                    image_similarity_results.append((img_path, similarity))

                # Print results
                for img_path, score in image_similarity_results:
                    print(f"Similarity with '{img_path}': {score:.2f}%")
            else:
                print("Reference image download failed; comparison will not be performed.")








            # Calculate percentages
            percentages = calculate_percentages(title_similarity_results, image_similarity_results)


            # Finding the highest percentage and its index
            highest_percentage = max(percentages)
            highest_index = percentages.index(highest_percentage)

            webuy_link = webuy_product_links[highest_index]
            amazon_driver.get(webuy_link)
            


            # Example usage
            clear_temp_images()
            time.sleep(2)
            for _ in range(0, 5):
                html_source = amazon_driver.page_source
                webuy_soup = BeautifulSoup(html_source, "html.parser")

                cookies = webuy_soup.find(id="onetrust-accept-btn-handler")
                cookies_path = '/html/body/div[3]/div[2]/div/div/div[2]/div/div/button'
                if cookies:
                    print("cookies found")
                    wait_for_element_to_be_clickable(amazon_driver, cookies_path).click()

                    break
                else:
                    time.sleep(1)

            for _ in range(0, 5):
                html_source = amazon_driver.page_source
                webuy_soup = BeautifulSoup(html_source, "html.parser")
                # Extracting the price
                if webuy_soup.find(class_="sell-price md-mb-0 d-block w-100") and webuy_soup.find(class_="heading-s-semibold md-heading-l-semibold")and webuy_soup.find(class_="product-gallery-image t058-checked"):
                    webuy_priceSTR = webuy_soup.find(class_="sell-price md-mb-0 d-block w-100").text.replace("£", "")
                    webuy_title = webuy_soup.find(class_="heading-s-semibold md-heading-l-semibold").text
                    webuy_image_link = webuy_soup.find(class_="product-gallery-image t058-checked").find("img")['src']

                    break        
                else:
                    time.sleep(1)
            webuy_image = download_webuy_image(webuy_image_link)
            # Converting the price to a float and adding 3
            webuy_price = float(webuy_priceSTR) + 3
            formatted_price = f"£{webuy_price:.2f}"

            
            if check_prices_more_than_webuyPrice(amazon_prices, (webuy_price*2)):
                print("invalid product")
                valid_product = False
                continue

            
            # Alternatively, using an f-string

            # Printing the formatted price
            print("The formatted price is:", formatted_price)
            for _ in range(0, 5):
                check_store_button_PATH = '/html/body/div[1]/div/main/div/div/div[1]/div[1]/div[2]/div[4]/div[2]/div/span'

                try:
                    # Wait until the element is clickable
                    WebDriverWait(amazon_driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, check_store_button_PATH))
                    )
                    amazon_driver.find_element(By.XPATH, check_store_button_PATH).click()
                except Exception as e:
                    time.sleep(0.5)

            for _ in range(0, 5):
                
                all_store_button_PATH = '/html/body/div[1]/div/main/div/div/div[6]/div[2]/button[2]/span'

                try:
                    # Wait until the element is clickable
                    WebDriverWait(amazon_driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, all_store_button_PATH))
                    )
                    amazon_driver.find_element(By.XPATH, all_store_button_PATH).click()
                except Exception as e:
                    time.sleep(0.5)
            html_source = amazon_driver.page_source
            webuy_soup = BeautifulSoup(html_source, "html.parser")
            store_name_class = 'store-name d-flex justify-content-between'
            for _ in range(0, 5):
                html_source = amazon_driver.page_source
                webuy_soup = BeautifulSoup(html_source, "html.parser")
                webuy_sellers_tag = webuy_soup.find(class_="store-list")
                if webuy_sellers_tag:
                    stores = webuy_sellers_tag.find_all(class_="store-card")
                    check_store_button_present = True
                else:
                    check_store_button_present = False
                
            if check_store_button_present == False or len(stores) < 3:
                valid_product = False
                continue
            import json
            file_path = "storage.json"
            # Initial new data values (replace these with actual data in your script)
            new_data = {
                'amazon_title': product_title,
                'amazon_link': link,
                'webuy_title': webuy_title,
                'webuy_link': webuy_link,
                'amazon_image': reference_image_path,
                'webuy_image': webuy_image,
                'calculated_price': calculated_price,
                'product_code': product_code,
                'webuy_price': float(f"{webuy_price:.2f}")
            }

            # Function to load the existing data from the JSON file (or create new lists if file doesn't exist)
            def load_existing_data(file_path):
                if os.path.exists(file_path):
                    with open(file_path, 'r') as file:
                        try:
                            return json.load(file)
                        except json.JSONDecodeError:
                            # If the file is empty or invalid JSON, start with empty lists
                            return {
                                'amazon_titles': [],
                                'amazon_links': [],
                                'webuy_titles': [],
                                'webuy_links': [],
                                'amazon_images': [],
                                'webuy_images': [],
                                'calculated_prices': [],
                                'product_codes': [],
                                'webuy_prices': []
                            }
                else:
                    # If the file doesn't exist, return empty lists
                    return {
                        'amazon_titles': [],
                        'amazon_links': [],
                        'webuy_titles': [],
                        'webuy_links': [],
                        'amazon_images': [],
                        'webuy_images': [],
                        'calculated_prices': [],
                        'product_codes': [],
                        'webuy_prices': []
                    }

            # Load the existing data
            existing_data = load_existing_data(file_path)

            # Check the conditions and append new data to each corresponding list
            if all([
                new_data['amazon_title'] is not None,
                new_data['amazon_link'] is not None,
                new_data['webuy_title'] is not None,
                new_data['webuy_link'] is not None,
                new_data['amazon_image'] is not None,
                new_data['webuy_image'] is not None,
                new_data['calculated_price'] is not None,
                new_data['product_code'] is not None,
                new_data['webuy_price'] is not None
            ]):
                existing_data['amazon_titles'].append(new_data['amazon_title'])
                existing_data['amazon_links'].append(new_data['amazon_link'])
                existing_data['webuy_titles'].append(new_data['webuy_title'])
                existing_data['webuy_links'].append(new_data['webuy_link'])
                existing_data['amazon_images'].append(new_data['amazon_image'])
                existing_data['webuy_images'].append(new_data['webuy_image'])
                existing_data['calculated_prices'].append(new_data['calculated_price'])
                existing_data['product_codes'].append(new_data['product_code'])
                existing_data['webuy_prices'].append(new_data['webuy_price'])

                # Save updated data back to the JSON file
                with open(file_path, 'w') as file:
                    json.dump(existing_data, file, indent=4)

                print(f"Data successfully added to {file_path}!")

            else:
                print("invalid product")
                valid_product = False
                continue
            break
        except Exception:
            print("invalid product")
            valid_product = False   
            continue






elemet found
'aria-label'
First whitelisted seller found at index: 0
No sellers in the list are blacklisted.
Price after reducing 1% from 57.79 is 57.2121
Similarity with 'Girls Und Panzer Collection': 84.38%
Similarity with 'Girls Und Panzer OVA Collection (12)': 76.71%
Libraries are installed correctly!
Similarity with 'images\temporary_images\5060067005498_l.jpg': 52.33%
Similarity with 'images\temporary_images\5060067005535_l.jpg': 30.42%
Deleted: images\temporary_images\5060067005498_l.jpg
Deleted: images\temporary_images\5060067005535_l.jpg
invalid product
elemet found
'aria-label'
First whitelisted seller found at index: 0
No sellers in the list are blacklisted.
Price after reducing 1% from 57.79 is 57.2121
Similarity with 'Gran Turismo (12) 2023 4K UHD+BR Steelbook (Gold)': 56.25%
Similarity with 'Gran Turismo (12) 2023 4K UHD+BR Steelbook (Blue)': 56.25%
Similarity with 'Mad Max Fury Road (15) 2015 w/ B&C Ed. 4K UHD+2BR - Ltd Ed. Steelbook': 35.14%
Similarity with 'Gran Turism

KeyboardInterrupt: 

In [76]:
amazon_sellers[0].strip().lower()

'selling-prime-uk'

In [77]:
print(amazon_sellers[0].strip().lower() == seller_white_list[0].strip().lower())

True


In [75]:
seller_white_list[0].strip().lower()

'selling-prime-uk'

In [34]:


# Read the data back from the file
with open('storage.json', 'r') as f:
    data = json.load(f)

# New variables for the lists to avoid overwriting the original ones
new_amazon_titles = data['amazon_titles']
new_amazon_links = data['amazon_links']
new_webuy_titles = data['webuy_titles']
new_webuy_links = data['webuy_links']
new_amazon_images = data['amazon_images']
new_webuy_images = data['webuy_images']
new_calculated_prices = data['calculated_prices']
new_product_codes = data['product_codes']
new_buy_prices = data['webuy_prices']


In [35]:
len(new_amazon_titles)

269

In [72]:
from openpyxl.styles import Alignment
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from openpyxl.styles import Font, PatternFill

# Define your data
data = {
    'Amazon_title': new_amazon_titles,
    'Amazon_link': new_amazon_links,
    'Webuy_title': new_webuy_titles,
    'Webuy_link': new_webuy_links,
    'Amazon_image': new_amazon_images,
    'Webuy_image': new_webuy_images,
    'Calculated_price': new_calculated_prices,
    'Buy_price': new_buy_prices,
    'Product_code': new_product_codes,
}

# Create a new workbook and select the active worksheet
wb = Workbook()
ws = wb.active
ws.title = "Products"

# Write headers in the specified order
headers = ['Amazon Title', 'Amazon Link', 'Webuy Title', 'Amazon Image', 'Webuy Image', 'Calculated Price', 'Product Code', 'Buy Price', 'Webuy Link']
ws.append(headers)

# Define styles for headers
header_fill = PatternFill(start_color="808080", end_color="808080", fill_type="solid")  # Gray fill
header_font = Font(bold=True)  # Bold font

# Apply styles to headers and align vertically
for col_num, header in enumerate(headers, start=1):
    cell = ws.cell(row=1, column=col_num)
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = Alignment(vertical='center')  # Vertically align headers

# Set column widths
column_widths = [30, 20, 30, 15, 15, 15, 15, 15, 35]  # Adjust widths as needed
for col_num, width in enumerate(column_widths, start=1):
    ws.column_dimensions[openpyxl.utils.get_column_letter(col_num)].width = width

# Add data and images
for i in range(len(data['Amazon_title'])):
    # Amazon Title
    amazon_title_cell = ws.cell(row=i + 2, column=1, value=data['Amazon_title'][i])
    amazon_title_cell.alignment = Alignment(vertical='center')  # Vertically align

    # Amazon Link (displays "Amazon Link" as plain text but clickable)
    amazon_link_cell = ws.cell(row=i + 2, column=2, value="Amazon Link")
    amazon_link_cell.hyperlink = data['Amazon_link'][i]  # Set the hyperlink
    amazon_link_cell.alignment = Alignment(vertical='center')  # Vertically align

    # Webuy Title
    webuy_title_cell = ws.cell(row=i + 2, column=3, value=data['Webuy_title'][i])
    webuy_title_cell.alignment = Alignment(vertical='center')  # Vertically align

    # Amazon Image in column 'D'
    amazon_img_path = data['Amazon_image'][i]
    amazon_img = Image(amazon_img_path)
    amazon_img.width = 50
    amazon_img.height = 50
    amazon_img.anchor = f'D{i + 2}'  # Set image in column D
    ws.add_image(amazon_img)

    # Webuy Image in column 'E'
    webuy_img_path = data['Webuy_image'][i]
    webuy_img = Image(webuy_img_path)
    webuy_img.width = 50
    webuy_img.height = 50
    webuy_img.anchor = f'E{i + 2}'  # Set image in column E
    ws.add_image(webuy_img)

    # Calculated Price
    calculated_price_cell = ws.cell(row=i + 2, column=6, value=f"£{data['Calculated_price'][i]:.2f}")
    calculated_price_cell.alignment = Alignment(vertical='center')  # Vertically align

    # Product Code
    product_code_cell = ws.cell(row=i + 2, column=7, value=data['Product_code'][i])
    product_code_cell.alignment = Alignment(vertical='center')  # Vertically align

    # Buy Price
    buy_price_cell = ws.cell(row=i + 2, column=8, value=f"£{data['Buy_price'][i]:.2f}")
    buy_price_cell.alignment = Alignment(vertical='center')  # Vertically align

    # Webuy Link (display as regular text, clickable, and copyable)
    webuy_link_cell = ws.cell(row=i + 2, column=9, value=data['Webuy_link'][i])
    webuy_link_cell.hyperlink = data['Webuy_link'][i]  # Hyperlink added
    webuy_link_cell.alignment = Alignment(vertical='center')  # Vertically align
    webuy_link_cell.font = Font(color="000000", underline=None)  # Regular black text without underline

    # Set row height to fit images
    ws.row_dimensions[i + 2].height = 50  # Ensure row height is set based on image size

# Save the workbook
wb.save('final_sheet_with_correct_order.xlsx')


In [25]:
with open("all_output_links.txt", "a") as file:
    for link in amazon_first_links3:
        file.write(link+ "\n")

In [64]:
webuy_images.append

['images\\temporary_images\\uk_badge.png',
 'images\\temporary_images\\uk_badge.png',
 'images\\temporary_images\\uk_badge.png',
 'images\\temporary_images\\uk_badge.png',
 'images\\temporary_images\\uk_badge.png',
 'images\\temporary_images\\uk_badge.png',
 'images\\temporary_images\\uk_badge.png',
 'images\\temporary_images\\uk_badge.png',
 'images\\temporary_images\\uk_badge.png',
 'images\\temporary_images\\uk_badge.png',
 'images\\temporary_images\\uk_badge.png',
 'images\\temporary_images\\uk_badge.png',
 'images\\temporary_images\\uk_badge.png',
 'images\\temporary_images\\uk_badge.png',
 'images\\temporary_images\\uk_badge.png',
 'images\\temporary_images\\uk_badge.png',
 'images\\webuy_images\\uk_badge.png',
 'images\\webuy_images\\uk_badge.png',
 'images\\webuy_images\\uk_badge.png',
 'images\\webuy_images\\5060139182584_l.jpg',
 'images\\webuy_images\\3455199427515_l.jpg',
 'images\\webuy_images\\018713553195_l.jpg',
 'images\\webuy_images\\5055298053813_l.jpg',
 'images\\we

In [55]:
file_path = "storage.json"
# Initial new data values (replace these with actual data in your script)
new_data = {
    'amazon_title': product_title,
    'amazon_link': link,
    'webuy_title': webuy_title,
    'webuy_link': webuy_link,
    'amazon_image': reference_image_path,
    'webuy_image': webuy_image,
    'calculated_price': calculated_price,
    'product_code': product_code,
    'webuy_price': float(f"{webuy_price:.2f}")
}

# Function to load the existing data from the JSON file (or create new lists if file doesn't exist)
def load_existing_data(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            try:
                return json.load(file)
            except json.JSONDecodeError:
                # If the file is empty or invalid JSON, start with empty lists
                return {
                    'amazon_titles': [],
                    'amazon_links': [],
                    'webuy_titles': [],
                    'webuy_links': [],
                    'amazon_images': [],
                    'webuy_images': [],
                    'calculated_prices': [],
                    'product_codes': [],
                    'webuy_prices': []
                }
    else:
        # If the file doesn't exist, return empty lists
        return {
            'amazon_titles': [],
            'amazon_links': [],
            'webuy_titles': [],
            'webuy_links': [],
            'amazon_images': [],
            'webuy_images': [],
            'calculated_prices': [],
            'product_codes': [],
            'webuy_prices': []
        }

# Load the existing data
existing_data = load_existing_data(file_path)

# Check the conditions and append new data to each corresponding list
if all([
    new_data['amazon_title'] is not None,
    new_data['amazon_link'] is not None,
    new_data['webuy_title'] is not None,
    new_data['webuy_link'] is not None,
    new_data['amazon_image'] is not None,
    new_data['webuy_image'] is not None,
    new_data['calculated_price'] is not None,
    new_data['product_code'] is not None,
    new_data['webuy_price'] is not None
]):
    existing_data['amazon_titles'].append(new_data['amazon_title'])
    existing_data['amazon_links'].append(new_data['amazon_link'])
    existing_data['webuy_titles'].append(new_data['webuy_title'])
    existing_data['webuy_links'].append(new_data['webuy_link'])
    existing_data['amazon_images'].append(new_data['amazon_image'])
    existing_data['webuy_images'].append(new_data['webuy_image'])
    existing_data['calculated_prices'].append(new_data['calculated_price'])
    existing_data['product_codes'].append(new_data['product_code'])
    existing_data['webuy_prices'].append(new_data['webuy_price'])

    # Save updated data back to the JSON file
    with open(file_path, 'w') as file:
        json.dump(existing_data, file, indent=4)


In [16]:
import json

# Your data dictionary
data = {
    'Amazon_title': amazon_titles,
    'Amazon_link': amazon_links,
    'Webuy_title': webuy_titles,
    'Webuy_link': webuy_links,
    'Amazon_image': amazon_images,
    'Webuy_image': webuy_images,
    'Calculated_price': calculated_prices,
    'Product_code': product_codes,
    'Buy_price': buy_prices,
}

# Write data to a JSON file
with open('data.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)  # indent=4 makes the JSON file pretty-printed


In [11]:
import json

# Function to read data from a file (assuming JSON format for simplicity)
def read_data_from_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Function to write data back to a file (if needed)
def write_data_to_file(data, file_path):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

# Original data (from file1)
data = read_data_from_file('file1_updated.json')

# Data from the second file (file2) with different key names
new_data = read_data_from_file('storage4.json')

# Mapping keys from new_data to match the original data format
key_mapping = {
    'amazon_titles': 'Amazon_title',
    'amazon_links': 'Amazon_link',
    'webuy_titles': 'Webuy_title',
    'webuy_links': 'Webuy_link',
    'amazon_images': 'Amazon_image',
    'webuy_images': 'Webuy_image',
    'calculated_prices': 'Calculated_price',
    'product_codes': 'Product_code',
    'webuy_prices': 'Buy_price'
}

# Append new data to the corresponding lists in the original dictionary
for new_key, original_key in key_mapping.items():
    if new_key in new_data and original_key in data:
        data[original_key].extend(new_data[new_key])

# Optionally, write the updated data back to file1.json or another file
write_data_to_file(data, 'file2_updated.json')

print("Data merging complete.")


Data merging complete.


In [59]:
html_source = webuy_driver.page_source
webuy_soup = BeautifulSoup(html_source, "html.parser")
webuy_soup.find(class_="product-gallery-image t058-checked").find("img")['src']

'https://uk.static.webuy.com/product_images/DVD/DVD Movies 1 Pound/5060033472040_l.jpg'

In [57]:
html_source = webuy_driver.page_source
webuy_soup = BeautifulSoup(html_source, "html.parser")
results = webuy_soup.find(class_='search-result-grid mb-s md-mb-0')

webuy_products = results.find_all(class_='search-product-card')

webuy_product_titles = []
webuy_product_links = []
webuy_product_images = []
for product in webuy_products:
    webuy_product_titles.append(product.find(class_='card-title').text)
    webuy_product_links.append('https://uk.webuy.com' + product.find('a')['href'])
    webuy_product_images.append(product.find('a').find("img")['src'])



In [58]:
webuy_product_images

['https://uk.static.webuy.com/product_images/DVD/DVD Movies 1 Pound/5060033472040_l.jpg',
 'https://uk.static.webuy.com/product_images/DVD/DVD Movies 1 Pound/671765910790_l.jpg']

In [17]:

other_sellers_button1 = '/html/body/div[2]/div/div[5]/div[4]/div[1]/div[5]/div[2]/div/div[2]/a/span'
other_sellers_button2 = 'buybox-see-all-buying-choices'
other_sellers_button3 = "/html/body/div[3]/div/div[8]/div[4]/div[1]/div[5]/div[2]/div/div[2]/a/span/span[1]"
other_sellers_button4 = "a-icon a-icon-arrow a-icon-small daodi-arrow-icon"
section_opened = False
other_sellers_button_PATH = 'a-price'

outofstock = False
for i in range(0,3):
    try: 
        html_source = amazon_driver.page_source
        soup = BeautifulSoup(html_source, "html.parser")
        check_stock = soup.find(id="outOfStock")
        if check_stock:
            outofstock = True
            break
        else:
            time.sleep(1)
    except Exception as e:
        print(e)
if outofstock:
    print("out")
    valid_product = False
    pass

element_found = False
try:
    # Wait until the element is clickable
    WebDriverWait(amazon_driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, other_sellers_button1))
    )
    amazon_driver.find_element(By.XPATH, other_sellers_button1).click()
    print("elemet found")
    element_found = True
    time.sleep(1)
    for i in range(0,3):
            try: 
                html_source = amazon_driver.page_source
                soup = BeautifulSoup(html_source, "html.parser")
                slider = soup.find(id="all-offers-display-scroller")
                if slider:
                    section_opened = True
                    break
                else:
                    time.sleep(1)
            except Exception as e:
                pass
except Exception as e:
    element_found = False
    print(f"Element not found")
    time.sleep(0.5)

if not element_found:
    other_sellers_button4 = "#dynamic-aod-ingress-box > div > div.a-section.a-spacing-none.daodi-content"
 
    try:
        # Wait until the element is clickable
        WebDriverWait(amazon_driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, other_sellers_button4))
        )
        amazon_driver.find_element(By.CSS_SELECTOR, other_sellers_button4).click()
        print("elemet found")
        element_found = True
        time.sleep(1)
        for i in range(0,3):
                try: 
                    html_source = amazon_driver.page_source
                    soup = BeautifulSoup(html_source, "html.parser")
                    slider = soup.find(id="all-offers-display-scroller")
                    if slider:
                        section_opened = True
                        break
                    else:
                        time.sleep(1)
                except Exception as e:
                    pass
    except Exception as e:
        element_found = False
        print(f"Element not found")
        time.sleep(0.5)

if not element_found:
    try:
        # Wait until the element is clickable
        WebDriverWait(amazon_driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, other_sellers_button3))
        )
        amazon_driver.find_element(By.XPATH, other_sellers_button3).click()
        print("elemet found")
        element_found = True
        time.sleep(1)
        for i in range(0,3):
                try: 
                    html_source = amazon_driver.page_source
                    soup = BeautifulSoup(html_source, "html.parser")
                    slider = soup.find(id="all-offers-display-scroller")
                    if slider:
                        section_opened = True
                        break
                    else:
                        time.sleep(1)
                except Exception as e:
                    pass
    except Exception as e:
        element_found = False
        print(f"Element not found")
        time.sleep(0.5)
if not section_opened:
    try:
            
        # Wait until the element is clickable
        WebDriverWait(amazon_driver, 10).until(
            EC.element_to_be_clickable((By.ID, other_sellers_button2))
        )
        amazon_driver.find_element(By.ID, other_sellers_button2).click()
        print("elemet found")
        time.sleep(1)
        section_opened = True
    except Exception:
        section_opened = False
        print("Element not found")

Element not found
elemet found


In [33]:
other_sellers_button_js_path = 'document.querySelector("#dynamic-aod-ingress-box > div > div.a-section.a-spacing-none.daodi-content > a > span > span:nth-child(1)")'

try:
    # Use JavaScript to find and click the element
    amazon_driver.execute_script(f'{other_sellers_button_js_path}.click();')
    print("Element clicked via JS path")
    element_found = True
    time.sleep(1)
    
    for i in range(3):
        try:
            html_source = amazon_driver.page_source
            soup = BeautifulSoup(html_source, "html.parser")
            slider = soup.find(id="all-offers-display-scroller")
            if slider:
                section_opened = True
                break
            else:
                time.sleep(1)
        except Exception as e:
            pass
except Exception as e:
    element_found = False
    print(f"Element not found or not clickable via JS path: {e}")
    time.sleep(0.5)


Element clicked via JS path


In [ ]:
import json

# Function to read data from a file (assuming JSON format)
def read_data_from_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Function to write data back to a file (if needed)
def write_data_to_file(data, file_path):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

# Load both dictionaries from file1 and file2
data1 = read_data_from_file('storage.json')
data2 = read_data_from_file('storage2.json')

# Merge data2 into data1 by appending the lists in each key
for key in data1:
    if key in data2:
        data1[key].extend(data2[key])

# Optionally, write the merged data to a new file
write_data_to_file(data1, 'merged_data.json')

print("Data merging complete.")
